In [13]:
!pip install plotly
!pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 3.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 KB 5.2 MB/s eta 0:00:00 MB/s eta 0:00:01


# Questions I want to answer
**1.** Which are the most sold devices? (rank)\
**2.** How much device's properties impact the sale price?\
**3.** What's the optimal sale price range?

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import re
import statsmodels

images_path = "Graph_images/"

In [2]:
dataset = pd.read_csv("apple_products.csv")
display(dataset.head(5))
dataset.shape

,Product Name,Product URL,Brand,Sale Price,Mrp,Discount Percentage,Number Of Ratings,Number Of Reviews,Upc,Star Rating,Ram
0,"APPLE iPhone 8 Plus (Gold, 64 GB)",https://www.flipkart.com/apple-iphone-8-plus-g...,Apple,49900,49900,0,3431,356,MOBEXRGV7EHHTGUH,4.6,2 GB
1,"APPLE iPhone 8 Plus (Space Grey, 256 GB)",https://www.flipkart.com/apple-iphone-8-plus-s...,Apple,84900,84900,0,3431,356,MOBEXRGVAC6TJT4F,4.6,2 GB
2,"APPLE iPhone 8 Plus (Silver, 256 GB)",https://www.flipkart.com/apple-iphone-8-plus-s...,Apple,84900,84900,0,3431,356,MOBEXRGVGETABXWZ,4.6,2 GB
3,"APPLE iPhone 8 (Silver, 256 GB)",https://www.flipkart.com/apple-iphone-8-silver...,Apple,77000,77000,0,11202,794,MOBEXRGVMZWUHCBA,4.5,2 GB
4,"APPLE iPhone 8 (Gold, 256 GB)",https://www.flipkart.com/apple-iphone-8-gold-2...,Apple,77000,77000,0,11202,794,MOBEXRGVPK7PFEJZ,4.5,2 GB


(62, 11)

In [3]:
# Removing useless columns
dataset.drop(columns = ["Product URL", "Brand", "Upc"], inplace = True)
dataset.shape

(62, 8)

In [4]:
dataset.isnull().sum()

Product Name           0
Sale Price             0
Mrp                    0
Discount Percentage    0
Number Of Ratings      0
Number Of Reviews      0
Star Rating            0
Ram                    0
dtype: int64

In [5]:
dataset.describe()

,Sale Price,Mrp,Discount Percentage,Number Of Ratings,Number Of Reviews,Star Rating
count,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000
mean,80073.887097,88058.064516,9.951613,22420.403226,1861.677419,4.575806
std,34310.446132,34728.825597,7.608079,33768.589550,2855.883830,0.059190
min,29999.000000,39900.000000,0.000000,542.000000,42.000000,4.500000
25%,49900.000000,54900.000000,6.000000,740.000000,64.000000,4.500000
50%,75900.000000,79900.000000,10.000000,2101.000000,180.000000,4.600000
75%,117100.000000,120950.000000,14.000000,43470.000000,3331.000000,4.600000
max,140900.000000,149900.000000,29.000000,95909.000000,8161.000000,4.700000


In [5]:
# Let's make Ram column useful by changing data type
dataset["Ram"] = (dataset["Ram"].apply(lambda x: x[0])).astype(int)
# Extract memory info
regx = re.compile("([0-9]*) GB")
dataset["Memory"] = dataset["Product Name"].apply(lambda x: re.search(regx, x).group(1)).astype(int)

dataset

,Product Name,Sale Price,Mrp,Discount Percentage,Number Of Ratings,Number Of Reviews,Star Rating,Ram,Memory
0,"APPLE iPhone 8 Plus (Gold, 64 GB)",49900,49900,0,3431,356,4.6,2,64
1,"APPLE iPhone 8 Plus (Space Grey, 256 GB)",84900,84900,0,3431,356,4.6,2,256
2,"APPLE iPhone 8 Plus (Silver, 256 GB)",84900,84900,0,3431,356,4.6,2,256
3,"APPLE iPhone 8 (Silver, 256 GB)",77000,77000,0,11202,794,4.5,2,256
4,"APPLE iPhone 8 (Gold, 256 GB)",77000,77000,0,11202,794,4.5,2,256
...,...,...,...,...,...,...,...,...,...
57,"APPLE iPhone SE (Black, 64 GB)",29999,39900,24,95909,8161,4.5,4,64
58,"APPLE iPhone 11 (Purple, 64 GB)",46999,54900,14,43470,3331,4.6,4,64
59,"APPLE iPhone 11 (White, 64 GB)",46999,54900,14,43470,3331,4.6,4,64
60,"APPLE iPhone 11 (Black, 64 GB)",46999,54900,14,43470,3331,4.6,4,64


## 1. Which are the most sold devices? (rank)

In [6]:
# Dataframe with top 10 rated devices
topDevices = dataset.sort_values(by = "Star Rating", ascending = False).head(10)
topDevices[["Product Name", "Number Of Ratings", "Star Rating"]]

,Product Name,Number Of Ratings,Star Rating
20,"APPLE iPhone 11 Pro Max (Midnight Green, 64 GB)",1078,4.7
17,"APPLE iPhone 11 Pro Max (Space Grey, 64 GB)",1078,4.7
16,"APPLE iPhone 11 Pro Max (Midnight Green, 256 GB)",1078,4.7
15,"APPLE iPhone 11 Pro Max (Gold, 64 GB)",1078,4.7
14,"APPLE iPhone 11 Pro Max (Gold, 256 GB)",1078,4.7
0,"APPLE iPhone 8 Plus (Gold, 64 GB)",3431,4.6
29,"APPLE iPhone 12 (White, 128 GB)",2101,4.6
32,"APPLE iPhone 12 Pro Max (Graphite, 128 GB)",580,4.6
35,"APPLE iPhone 12 (Black, 128 GB)",2101,4.6
36,"APPLE iPhone 12 (Blue, 128 GB)",2101,4.6


In [7]:
fig = px.bar(topDevices[["Product Name", "Number Of Ratings"]].sort_values(by = "Number Of Ratings", ascending = False), 
             x = "Product Name", 
             y = "Number Of Ratings")
fig.update_layout(title = "Number of rating per iPhone model")
fig.write_html(images_path + "RatingsPerIphoneModel.html")
fig.show()

In [8]:
# Normalize star rating by number of ratings received. 
# Let's see if the rank order changes
df = topDevices[["Product Name", "Number Of Ratings", "Star Rating"]].copy()
df["Star Rating Normalized"] = df["Star Rating"] / df["Number Of Ratings"]
df.sort_values(by = "Star Rating Normalized", ascending = True)

,Product Name,Number Of Ratings,Star Rating,Star Rating Normalized
0,"APPLE iPhone 8 Plus (Gold, 64 GB)",3431,4.6,0.001341
29,"APPLE iPhone 12 (White, 128 GB)",2101,4.6,0.002189
35,"APPLE iPhone 12 (Black, 128 GB)",2101,4.6,0.002189
36,"APPLE iPhone 12 (Blue, 128 GB)",2101,4.6,0.002189
20,"APPLE iPhone 11 Pro Max (Midnight Green, 64 GB)",1078,4.7,0.004360
17,"APPLE iPhone 11 Pro Max (Space Grey, 64 GB)",1078,4.7,0.004360
16,"APPLE iPhone 11 Pro Max (Midnight Green, 256 GB)",1078,4.7,0.004360
15,"APPLE iPhone 11 Pro Max (Gold, 64 GB)",1078,4.7,0.004360
14,"APPLE iPhone 11 Pro Max (Gold, 256 GB)",1078,4.7,0.004360
32,"APPLE iPhone 12 Pro Max (Graphite, 128 GB)",580,4.6,0.007931


## 2. How much device's properties impact the sale price?

In [9]:
# Relationship between dimensions of the device and sale price
fig = px.scatter(dataset,
                 x = "Memory",
                 y = "Ram",
                 color = "Sale Price",
                 trendline = "ols")
fig.update_traces(marker = {"size" : 15})
fig.update_layout(title = "Bigger the device, higher the price?",
                  showlegend = True)
fig.write_html(images_path + "BiggerDeviceHigherPrice.html")
fig.show()

## 3. What's the optimal sale price range?

In [10]:
fig = px.scatter(dataset,
                 x = "Number Of Ratings",
                 y = "Sale Price",
                 trendline = "ols")
fig.update_traces(marker = {"size" : 10})
fig.update_layout(title = "Lower the price, easier to sell?")
fig.write_html(images_path + "LowerPriceEasierSale.html")
fig.show()